# Testing MCTS with Various Models
Here we will test MCTS first with the DKT model

In [1]:
#%load_ext autoreload
#%autoreload 2
#%reload_ext autoreload

from concept_dependency_graph import ConceptDependencyGraph
import data_generator as dg
from student import Student
import simple_mdp as sm

In [2]:
n_concepts = 5
learn_prob = 0.15
n_students = 100000
seqlen = 40
policy = 'random'
filename = 'test-n{}-l{}-lp{}-{}.pickle'.format(n_students, seqlen, int(learn_prob*100), policy)
concept_tree = sm.create_custom_dependency()
test_student = Student(p_trans_satisfied=learn_prob, p_trans_not_satisfied=0.0, p_get_ex_correct_if_concepts_learned=1.0)
print(filename)

test-n100000-l40-lp15-random.pickle


In [3]:
print ("Initializing synthetic data sets...")
dg.generate_data(concept_tree, student=test_student, n_students=n_students, seqlen=seqlen, policy=policy, filename="{}{}".format(dg.SYN_DATA_DIR, filename))
print ("Data generation completed. ")

Initializing synthetic data sets...
Generating data for 100000 students with behavior policy random and sequence length 4.
Data generation completed. 


In [3]:
import dynamics_model_class as dmc
import numpy as np
import dataset_utils

In [4]:
# test_model hidden=16
# test_model_small hidden=5
# test_model_mid hidden=10
dmodel = dmc.DynamicsModel(model_id="test_model_mid", timesteps=seqlen, load_checkpoint=False)

Loading RNN dynamics model...
Directory path for tensorboard summaries: ../tensorboard_logs/test_model_mid/
Checkpoint directory path: ../checkpoints/test_model_mid/
Model loaded.


In [5]:
# load toy data
data = dataset_utils.load_data(filename='{}{}'.format(dg.SYN_DATA_DIR, filename))
input_data_, output_mask_, target_data_ = dataset_utils.preprocess_data_for_rnn(data)

In [6]:
train_data = (input_data_[:,:,:], output_mask_[:,:,:], target_data_[:,:,:])

In [8]:
dmodel.train(train_data, n_epoch=2, load_checkpoint=True)

Training Step: 9848  | total loss: 0.00899 | time: 35.644s
| Adam | epoch: 007 | loss: 0.00899 -- iter: 89984/90000
Training Step: 9849  | total loss: 0.00890 | time: 37.228s
| Adam | epoch: 007 | loss: 0.00890 | val_loss: 0.00872 -- iter: 90000/90000
--
Type is unsupported, or the types of the items don't match field type in CollectionDef.
'list' object has no attribute 'name'
Type is unsupported, or the types of the items don't match field type in CollectionDef.
'list' object has no attribute 'name'


In [29]:
preds = dmodel.predict(input_data_[:1,:2, :])
print len(preds)
print len(preds[0])
print preds[0]

1
4
[[0.7310136556625366, 0.1430114209651947, 0.14930900931358337, 0.0036982132587581873, 0.003802887862548232], [0.8807896375656128, 0.1483885645866394, 0.15391851961612701, 0.00020966221927665174, 0.0002947214525192976], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0]]


In [30]:
rnnsim = dmc.RnnStudentSim(dmodel)

In [31]:
from mcts import *
actix = 0
actvec = np.zeros((n_concepts,))
actvec[actix] = 1
act = StudentAction(actix, actvec)
print rnnsim.sequence
print rnnsim.sample_observations()

[]
None


In [32]:
rnnsim.advance_simulator(act, 1)


In [33]:
print rnnsim.sequence
print rnnsim.sample_observations()
print rnnsim.sample_reward()

[array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])]
[0.7310136556625366, 0.1430114209651947, 0.14930900931358337, 0.0036982132587581873, 0.003802887862548232]
1.03083518706
